# Projet 7 - Réalisez une analyse de sentiments grâce au Deep Learning

> 🎓 OpenClassrooms • Parcours [AI Engineer](https://openclassrooms.com/fr/paths/795-ai-engineer) | 👋 *Étudiant* : [David Scanu](https://www.linkedin.com/in/davidscanu14/)

## Partie 2 : Approches classiques de Machine Learning

Ce notebook implémente plusieurs modèles de **machine learning traditionnels pour la classification de sentiment des tweets**. Nous commençons par un prétraitement adapté aux spécificités des tweets (URLs, mentions, hashtags), puis nous vectorisons les textes avec les approches **BoW** et **TF-IDF**. Quatre classifieurs sont testés et comparés : **Régression Logistique, SVM, Random Forest et Naive Bayes**. Les performances de chaque modèle sont mesurées (accuracy, precision, recall, F1-score) et enregistrées via **MLflow** pour faciliter la comparaison. Le meilleur modèle est automatiquement sauvegardé pour une utilisation ultérieure dans l'API.

## 📝 Contexte

Dans le cadre de ma formation d'AI Engineer chez OpenClassrooms, ce projet s'inscrit dans un scénario professionnel où j'interviens en tant qu'ingénieur IA chez MIC (Marketing Intelligence Consulting), entreprise de conseil spécialisée en marketing digital.

Notre client, Air Paradis (compagnie aérienne), souhaite **anticiper les bad buzz sur les réseaux sociaux**. La mission consiste à développer un produit IA permettant de **prédire le sentiment associé à un tweet**, afin d'améliorer la gestion de sa réputation en ligne.

## ⚡ Mission

> Développer un modèle d'IA permettant de prédire le sentiment associé à un tweet.

Créer un prototype fonctionnel d'un modèle d'**analyse de sentiments pour tweets** selon trois approches différentes :

1. **Modèle sur mesure simple** : Approche classique (régression logistique) pour une prédiction rapide
2. **Modèle sur mesure avancé** : Utilisation de réseaux de neurones profonds avec différents word embeddings
3. **Modèle avancé BERT** : Exploration de l'apport en performance d'un modèle BERT

Cette mission implique également la mise en œuvre d'une démarche MLOps complète :
- Utilisation de MLFlow pour le tracking des expérimentations et le stockage des modèles
- Création d'un pipeline de déploiement continu (Git + Github + plateforme Cloud)
- Intégration de tests unitaires automatisés
- Mise en place d'un suivi de performance en production via Azure Application Insight

## 🗓️ Plan de travail

1. **Exploration et préparation des données**
   - Acquisition des données de tweets Open Source
   - Analyse exploratoire et prétraitement des textes

2. **Développement des modèles**
   - Implémentation du modèle classique (régression logistique)
   - Conception du modèle avancé avec différents word embeddings
   - Test du modèle BERT pour l'analyse de sentiments
   - Comparaison des performances via MLFlow

3. **Mise en place de la démarche MLOps**
   - Configuration de MLFlow pour le tracking des expérimentations
   - Création du dépôt Git avec structure de projet appropriée
   - Implémentation des tests unitaires automatisés
   - Configuration du pipeline de déploiement continu

4. **Déploiement et monitoring**
   - Développement de l'API de prédiction avec FastAPI
   - Déploiement sur Heroku
   - Création de l'interface de test (Streamlit ou Next.js)
   - Configuration du suivi via Azure Application Insight

5. **Communication**
   - Rédaction de l'article de blog
   - Préparation du support de présentation

## Importation des bibliothèques

In [77]:
# Importations nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import time
import warnings
from collections import Counter
import pickle
from tqdm import tqdm

# Importations NLTK
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Importations scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             fbeta_score, make_scorer, matthews_corrcoef, balanced_accuracy_score,
                             classification_report, confusion_matrix, roc_auc_score, roc_curve)

# Configuration des visualisations
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.family'] = 'DejaVu Sans'
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt_tab to /home/david/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/david/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 💾 Jeu de données : Sentiment140

Le jeu de données [Sentiment140 dataset with 1.6 million tweets](https://www.kaggle.com/datasets/kazanova/sentiment140) est une ressource majeure pour l'analyse de sentiment sur Twitter, comprenant **1,6 million de tweets** extraits via l'API Twitter. Ces tweets ont été automatiquement annotés selon leur polarité sentimentale, offrant une base solide pour développer des modèles de classification de sentiment.

Le jeu de données est organisé en 6 colonnes distinctes :

1. **target** : La polarité du sentiment exprimé dans le tweet.
   - 0 = sentiment négatif
   - 2 = sentiment neutre
   - 4 = sentiment positif
2. **ids** : L'identifiant unique du tweet (exemple : *2087*)
3. **date** : La date et l'heure de publication du tweet.
4. **flag** : La requête utilisée pour obtenir le tweet.
   - Exemple : *lyx*
   - Si aucune requête n'a été utilisée : *NO_QUERY*
5. **user** : Le nom d'utilisateur de l'auteur du tweet.
6. **text** : Le contenu textuel du tweet.

In [2]:
%%time 

# Define the URL and the local file path
url = "https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+7%C2%A0-+D%C3%A9tectez+les+Bad+Buzz+gr%C3%A2ce+au+Deep+Learning/sentiment140.zip"
local_zip_path = "./content/data/sentiment140.zip"
extract_path = "./content/data"

if not os.path.exists(extract_path):

    # Create the directory if it doesn't exist
    os.makedirs(extract_path, exist_ok=True)

    # Download the zip file
    response = requests.get(url)
    with open(local_zip_path, 'wb') as file:
        file.write(response.content)

    # Extract the contents of the zip file
    with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    # Delete the zip file
    os.remove(local_zip_path)

CPU times: user 50 μs, sys: 5 μs, total: 55 μs
Wall time: 60.3 μs


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', True)

In [4]:

# Define the path to the CSV file
csv_file_path = os.path.join(extract_path, 'training.1600000.processed.noemoticon.csv')

# Define the column names
column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']

# Load the dataset into a pandas DataFrame
raw_data = pd.read_csv(csv_file_path, encoding='latin-1', names=column_names)

# Display the first few rows of the DataFrame
raw_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
print(f"Ce dataframe contient {raw_data.shape[0]} lignes et {raw_data.shape[1]} colonnes.")

Ce dataframe contient 1600000 lignes et 6 colonnes.


## Recommandations de modélisation

Approches recommandées pour l'analyse de sentiment:

1. Approches classiques de Machine Learning:
   - Modèles basés sur les sacs de mots (BoW) ou TF-IDF avec classifieurs comme Régression Logistique, SVM, Random Forest ou Naive Bayes

2. Word Embeddings + Deep Learning (2 modèles):
   - Utiliser des embeddings pré-entraînés (Word2Vec, GloVe, FastText) avec des classifieurs Deep Learning
   - Utiliser des réseaux de neurones récurrents (RNN, **LSTM**, GRU)

3. Modèles transformers (BERT, Sentence Transformers, RoBERTa, DistilBERT):
   - Fine-tuning de modèles pré-entraînés spécifiques à Twitter comme BERTweet

Considérations importantes:

1. Déséquilibre des classes: utiliser des techniques comme SMOTE, sous-échantillonnage, ou pondération des classes
2. Validation croisée: essentielle pour évaluer correctement les performances
3. Métriques d'évaluation: ne pas se limiter à l'accuracy, utiliser F1-score, précision, rappel, et AUC-ROC
4. Interprétabilité: pour certaines applications, privilégier des modèles interprétables ou utiliser SHAP/LIME
5. Dépendance temporelle: considérer l'évolution du langage sur Twitter au fil du temps

**Notes du mentor :**

Voici le texte extrait de cette capture d'écran :

- Création de deux modèles de Deep Learning, dont au moins un avec un layer LSTM.
- Simulation selon deux techniques de pré-traitement (lemmatization, stemming) sur l'un des 2 modèles, afin de choisir la technique pour la suite des simulations.
- Simulation selon 2 approches de word embedding (parmi Word2VEc, Glove, FastText), entraînés avec le jeu de données ou pré-entraînés sur au moins un des 2 modèles de Deep Learning, afin de choisir l'embedding pour la suite des simulations.
- Création ensuite d'un modèle BERT, il y a 2 approches possibles :
  - Générer des features (sentence embedding) à partir d'un TFBertModel (Hugging Face) ou d'un d'un model via le Hub TensorFlow, puis ajouter une ou des couches de classification
  - Utiliser directement un modèle Hugging Face de type TFBertForSequenceClassification
- En option tester USE (Universal Sentence Encoding) pour le feature engineering

Problèmes et erreurs courants :
- Temps de traitement et limitation de ressources en TensorFlow-Keras.
- Inspirer des exemples de modèles cités en ressources


## Approches classiques de Machine Learning

Notre démarche pour la classification de sentiment avec des approches classiques comprend:

1. **Prétraitement des tweets**
   - Nettoyage: suppression des caractères spéciaux
   - Tokenisation et lemmatisation
   - Remplacement des URLs et mentions par des tokens spéciaux

2. **Vectorisation du texte**
   - **Sac de mots (BoW)** : représentation basée sur la fréquence d'apparition
   - **TF-IDF** : pondération par importance relative des mots

3. **Modèles testés**
   - Régression Logistique: rapide et interprétable
   - SVM Linéaire: efficace pour les textes
   - Random Forest: robuste aux outliers
   - Naive Bayes: performant pour les classifications textuelles

4. **Évaluation et suivi**
   - Métriques: accuracy, précision, recall, F1-score
   - Tracking avec MLflow pour la reproductibilité et la comparaison

In [24]:
def convert_sentiment_label(df):
    converted_target_data = df.copy()
    converted_target_data['target'] = converted_target_data['target'].apply(lambda x: 0 if x == 0 else 1)
    return converted_target_data

converted_target_data = convert_sentiment_label(raw_data)

In [25]:
converted_target_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [34]:
def downsample_data(df, n_samples=50000):
    """
    Réduit la taille d'un DataFrame en échantillonnant aléatoirement un nombre spécifié de lignes pour chaque classe.
    """
    negative_samples = df[df['target'] == 0].sample(n=n_samples, random_state=42)
    positive_samples = df[df['target'] == 1].sample(n=n_samples, random_state=42)
    downsampled_data = pd.concat([negative_samples, positive_samples])
    return downsampled_data

downsampled_data = downsample_data(converted_target_data, n_samples=50000)
downsampled_data['target'].value_counts()

target
0    50000
1    50000
Name: count, dtype: int64

## Prétraitement

In [33]:
from multiprocessing import Pool

# Fonction de prétraitement pour les tweets
def preprocess_tweet(tweet):
    """
    Prétraite un tweet en appliquant plusieurs transformations :
    - Conversion en minuscules
    - Remplacement des URLs, mentions et hashtags par des tokens spéciaux
    - Suppression des caractères spéciaux
    - Tokenisation et lemmatisation
    - Suppression des stopwords
    """
    # Vérifier si le tweet est une chaîne de caractères
    if not isinstance(tweet, str):
        return ""
    
    # Convertir en minuscules
    tweet = tweet.lower()
    
    # Remplacer les URLs par un token spécial
    tweet = re.sub(r'https?://\S+|www\.\S+', '<URL>', tweet)
    
    # Remplacer les mentions par un token spécial
    tweet = re.sub(r'@\w+', '<MENTION>', tweet)
    
    # Traiter les hashtags (conserver le # comme token séparé et le mot qui suit)
    tweet = re.sub(r'#(\w+)', r'# \1', tweet)
    
    # Supprimer les caractères spéciaux et les nombres, mais garder les tokens spéciaux
    tweet = re.sub(r'[^\w\s<>@#!?]', '', tweet)
    
    # Tokenisation
    tokens = word_tokenize(tweet)
    
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Supprimer les stopwords, mais conserver les négations importantes
    stop_words = set(stopwords.words('english'))
    important_words = {'no', 'not', 'nor', 'neither', 'never', 'nobody', 'none', 'nothing', 'nowhere'}
    stop_words = stop_words - important_words
    tokens = [token for token in tokens if token not in stop_words]
    
    # Rejoindre les tokens en une chaîne
    return ' '.join(tokens)


def process_in_parallel(df, func, n_jobs=4):
    """
    Applique une fonction à un DataFrame en le divisant en parties et en traitant chaque partie en parallèle.
    Permet d'accélérer le traitement sur les ordinateurs multi-coeurs.
    """
    df_split = np.array_split(df, n_jobs)
    pool = Pool(n_jobs)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def apply_preprocessing(df_part):
    df_part['processed_text'] = df_part['text'].apply(preprocess_tweet)
    return df_part

In [37]:
# Appliquer le prétraitement à tous les tweets
print("Prétraitement des tweets en cours...")
# downsampled_data['processed_text'] = downsampled_data['text'].apply(preprocess_tweet)
# Utilisation
preprocessed_data = process_in_parallel(downsampled_data, apply_preprocessing, n_jobs=8)
print("Prétraitement terminé !")

Prétraitement des tweets en cours...


Prétraitement terminé !


In [38]:
preprocessed_data.head()

,target,ids,date,flag,user,text,processed_text
212188,0,1974671194,Sat May 30 13:36:31 PDT 2009,NO_QUERY,simba98,@xnausikaax oh no! where did u order from? tha...,< MENTION > oh no ! u order ? thats horrible
299036,0,1997882236,Mon Jun 01 17:37:11 PDT 2009,NO_QUERY,Seve76,A great hard training weekend is over. a coup...,great hard training weekend couple day rest le...
475978,0,2177756662,Mon Jun 15 06:39:05 PDT 2009,NO_QUERY,x__claireyy__x,"Right, off to work Only 5 hours to go until I...",right work 5 hour go im free xd
588988,0,2216838047,Wed Jun 17 20:02:12 PDT 2009,NO_QUERY,Balasi,I am craving for japanese food,craving japanese food
138859,0,1880666283,Fri May 22 02:03:31 PDT 2009,NO_QUERY,djrickdawson,Jean Michel Jarre concert tomorrow gotta work...,jean michel jarre concert tomorrow got ta work...


## Division des données en ensembles d'entraînement et de test

In [39]:
# Diviser les données en ensembles d'entraînement et de test
from sklearn.model_selection import train_test_split

X = preprocessed_data['processed_text']
y = preprocessed_data['target']

# Division train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Taille de l'ensemble d'entraînement: {X_train.shape[0]} exemples")
print(f"Taille de l'ensemble de test: {X_test.shape[0]} exemples")

Taille de l'ensemble d'entraînement: 80000 exemples
Taille de l'ensemble de test: 20000 exemples


## Entraînement des modèles et tracking avec MLflow

### Configuration de MLFlow

In [58]:
import mlflow
from mlflow.models import infer_signature
from mlflow import MlflowClient
from dotenv import load_dotenv

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

# Configuration de MLflow avec les variables d'environnement
mlflow_tracking_uri = os.getenv("MLFLOW_TRACKING_URI")
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")

# Configuration explicite de MLflow
mlflow.set_tracking_uri(mlflow_tracking_uri)
print(f"MLflow Tracking URI: {mlflow_tracking_uri}")

# Configuration explicite des identifiants AWS
os.environ["AWS_ACCESS_KEY_ID"] = aws_access_key_id
os.environ["AWS_SECRET_ACCESS_KEY"] = aws_secret_access_key
print("Identifiants AWS configurés")

MLflow Tracking URI: https://zany-orbit-q59ppqxj6j34j4x-5001.app.github.dev/
Identifiants AWS configurés


In [69]:
# Créer l'expérience MLflow
mlflow.set_experiment("OC Projet 7")

<Experiment: artifact_location='s3://mlflow-artefact-store/models/52', creation_time=1741945848821, experiment_id='52', last_update_time=1741945848821, lifecycle_stage='active', name='OC Projet 7', tags={}>

### Vectorisation des textes

La vectorisation est le processus qui transforme des documents textuels en vecteurs numériques exploitables par les algorithmes de machine learning. Les deux approches simples sont :

- **Bag of Words (BoW)** : compte simplement la fréquence d'apparition de chaque mot
- **TF-IDF** : pondère les mots selon leur importance (fréquence dans le document / fréquence dans tous les documents)

Ces méthodes créent des matrices généralement très creuses (>99% de zéros) car chaque document n'utilise qu'une petite partie du vocabulaire total. Dans notre implémentation, nous utilisons `CountVectorizer` pour l'approche BoW et `TfidfVectorizer` pour le TF-IDF, avec des n-grammes (1,2) qui permettent de capturer des expressions de deux mots consécutifs.


In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Créer les vectoriseurs
tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
bow_vectorizer = CountVectorizer(max_features=10000, ngram_range=(1, 2))


# Transformer les textes en vecteurs TF-IDF
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Transformer les textes en vecteurs BoW
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

print("Vectorisation terminée !")

Vectorisation terminée !


### Fonction d'évaluation des modèles

Dans le contexte d'un outil préventif de **détection de bad buzz pour Air Paradis**, il est probablement plus important de **ne pas manquer de tweets négatifs** (priorité au rappel), nous pouvons considérer le **F2-score** qui donne plus de poids au rappel qu'à la précision.

In [74]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name, vectorizer_name):
    """
    Évalue un modèle déjà entraîné et retourne les résultats
    """
    # Prédictions sur l'ensemble de test
    y_pred = model.predict(X_test)
    
    # Obtenir les probabilités ou scores de décision pour ROC AUC
    if hasattr(model, 'predict_proba'):
        y_score = model.predict_proba(X_test)[:, 1]
    elif hasattr(model, 'decision_function'):
        y_score = model.decision_function(X_test)
    else:
        y_score = y_pred  # Fallback pour les modèles qui n'ont pas ces méthodes
    
    # Calculer les métriques
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    f2 = fbeta_score(y_test, y_pred, beta=2)  # Ajout du F2-score
    roc_auc = roc_auc_score(y_test, y_score)
    
    # Créer la matrice de confusion sous forme de figure
    cm = confusion_matrix(y_test, y_pred)
    fig_cm, ax_cm = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax_cm)
    ax_cm.set_xlabel('Prédiction')
    ax_cm.set_ylabel('Valeur réelle')
    ax_cm.set_title(f'Matrice de confusion - {model_name} avec {vectorizer_name}')
    
    # Créer la courbe ROC
    fpr, tpr, _ = roc_curve(y_test, y_score)
    fig_roc, ax_roc = plt.subplots(figsize=(8, 6))
    ax_roc.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.3f})')
    ax_roc.plot([0, 1], [0, 1], 'k--')
    ax_roc.set_xlim([0.0, 1.0])
    ax_roc.set_ylim([0.0, 1.05])
    ax_roc.set_xlabel('Taux de faux positifs')
    ax_roc.set_ylabel('Taux de vrais positifs')
    ax_roc.set_title(f'Courbe ROC - {model_name} avec {vectorizer_name}')
    ax_roc.legend(loc="lower right")
    ax_roc.grid(True)
    
    # Afficher les résultats
    print(f"\nRésultats pour {model_name} avec {vectorizer_name}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"F2 Score: {f2:.4f}")  # Ajout du F2-score
    print(f"ROC AUC: {roc_auc:.4f}")
    print("\nMatrice de confusion:")
    print(cm)
    print("\nRapport de classification:")
    print(classification_report(y_test, y_pred))
    
    return accuracy, precision, recall, f1, f2, roc_auc, fig_cm, fig_roc, y_pred

### Initialiser les modèles à tester

Voici la liste des modèles que nous allons tester :

- Régression Logistique
- SVM Linéaire
- Forêt Aléatoire
- Naive Bayes

In [73]:
# Initialiser les modèles à tester
base_models = {
    "Regression_Logistique": LogisticRegression(random_state=42),
    "SVM_Lineaire": LinearSVC(random_state=42),
    "Random_Forest": RandomForestClassifier(random_state=42),
    "Naive_Bayes": MultinomialNB()
}

# Définir les grilles d'hyperparamètres pour GridSearchCV
param_grids = {
    "Regression_Logistique": {
        'C': [0.01, 0.1, 1.0, 10.0],
        'max_iter': [1000],
        'solver': ['liblinear', 'saga']
    },
    "SVM_Lineaire": {
        'C': [0.01, 0.1, 1.0, 10.0],
        'max_iter': [1000],
        'dual': [True, False]
    },
    "Random_Forest": {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5]
    },
    "Naive_Bayes": {
        'alpha': [0.1, 0.5, 1.0, 2.0]
    }
}

# Dictionnaire des vectoriseurs
vectorizers = {
    "TF-IDF": (tfidf_vectorizer, X_train_tfidf, X_test_tfidf),
    "BoW": (bow_vectorizer, X_train_bow, X_test_bow)
}

# Définir le scorer F2
scorers = {
    'f2': make_scorer(fbeta_score, beta=2),
    'f1': make_scorer(f1_score),
    'mcc': make_scorer(matthews_corrcoef),
    'balanced_acc': make_scorer(balanced_accuracy_score)
}

# Dictionnaire pour stocker les résultats
results = []

In [ ]:
# Tester chaque modèle avec les deux types de vectorisation
total_iterations = len(base_models) * len(vectorizers)
progress_bar = tqdm(total=total_iterations, desc="Progression globale")

# Tester chaque modèle avec les deux types de vectorisation
for model_name, base_model in base_models.items():
    for vectorizer_name, (vectorizer, X_train_vec, X_test_vec) in vectorizers.items():
        print(f"\n{'='*80}")
        print(f"Démarrage de GridSearchCV pour {model_name} avec {vectorizer_name}...")
        
        # Définir la grille de paramètres et créer GridSearchCV
        param_grid = param_grids[model_name]

        grid_search = GridSearchCV(
            base_model,
            param_grid,
            cv=5,
            scoring=scorers,
            refit='f2',
            n_jobs=-1,
            verbose=1,
            return_train_score=True
        )
        
        # Démarrer le run MLflow
        with mlflow.start_run(run_name=f"Modele_Simple_{model_name}_{vectorizer_name}"):
            # Journaliser les paramètres généraux
            mlflow.log_param("model_type", model_name)
            mlflow.log_param("vectorizer_type", vectorizer_name)
            mlflow.log_param("dataset_size", X_train.shape[0] + X_test.shape[0])
            mlflow.log_param("train_size", X_train.shape[0])
            mlflow.log_param("test_size", X_test.shape[0])
            mlflow.log_param("max_features", 10000)
            mlflow.log_param("ngram_range", "(1, 2)")
            mlflow.log_param("scoring_metric", "f2_score")  # Noter que F2 est utilisé
            
            # Mesurer le temps d'entraînement
            start_time = time.time()
            
            # Entraîner avec GridSearchCV
            grid_search.fit(X_train_vec, y_train)
            
            # Calculer le temps d'entraînement
            training_time = time.time() - start_time
            
            # Journaliser le temps d'entraînement
            mlflow.log_metric("training_time", training_time)
            
            # Récupérer et journaliser les meilleurs paramètres
            best_params = grid_search.best_params_
            for param, value in best_params.items():
                mlflow.log_param(f"best_{param}", value)
            
            # Journaliser le meilleur score de validation croisée
            mlflow.log_metric("best_cv_f2_score", grid_search.best_score_)
            
            # Récupérer le meilleur modèle
            best_model = grid_search.best_estimator_
            
            # Évaluer le meilleur modèle sur l'ensemble de test
            acc, prec, rec, f1, f2, roc_auc, fig_cm, fig_roc, y_pred = evaluate_model(
                best_model, X_train_vec, X_test_vec, y_train, y_test, model_name, vectorizer_name
            )
            
            # Journaliser les métriques
            mlflow.log_metric("accuracy", acc)
            mlflow.log_metric("precision", prec)
            mlflow.log_metric("recall", rec)
            mlflow.log_metric("f1", f1)
            mlflow.log_metric("f2", f2)
            mlflow.log_metric("roc_auc", roc_auc)
            
            # Journaliser les figures
            mlflow.log_figure(fig_cm, "confusion_matrix.png")
            mlflow.log_figure(fig_roc, "roc_curve.png")
            plt.close(fig_cm)
            plt.close(fig_roc)
            
            # Journaliser le modèle
            signature = infer_signature(X_train_vec, y_pred)
            mlflow.sklearn.log_model(best_model, "model", signature=signature)
            
            # Sauvegarder et journaliser le vectoriseur
            vectorizer_path = f"vectorizer_{vectorizer_name}.pkl"
            with open(vectorizer_path, "wb") as f:
                pickle.dump(vectorizer, f)
            mlflow.log_artifact(vectorizer_path)
            
            # Journaliser les résultats détaillés de GridSearchCV
            cv_results = pd.DataFrame(grid_search.cv_results_)
            cv_results_path = "cv_results.csv"
            cv_results.to_csv(cv_results_path, index=False)
            mlflow.log_artifact(cv_results_path)
            
            # Créer et journaliser un graphique des résultats de GridSearchCV
            plt.figure(figsize=(12, 8))
            params = [f"{k}={v}" for k, v in best_params.items()]
            params_str = ", ".join(params)
            
            # Extraire les scores moyens pour chaque paramètre principal
            for param in param_grid.keys():
                if len(param_grid[param]) > 1:  # Seulement si le paramètre a plusieurs valeurs
                    param_name = f"param_{param}"
                    if param_name in cv_results.columns:
                        # Utiliser la colonne spécifique à la métrique f2 (que vous avez définie comme principale)
                        scores_df = cv_results[[param_name, "mean_test_f2", "std_test_f2"]]
                        scores_df = scores_df.sort_values(param_name)
                        
                        plt.figure(figsize=(10, 6))
                        plt.errorbar(
                            scores_df[param_name].astype(str),
                            scores_df["mean_test_f2"],
                            yerr=scores_df["std_test_f2"],
                            fmt='-o'
                        )
                        plt.title(f'Scores de validation croisée (F2) par {param}')
                        plt.xlabel(param)
                        plt.ylabel('F2 Score moyen')
                        plt.grid(True)
                        mlflow.log_figure(plt.gcf(), f"cv_results_{param}.png")
                        plt.close()
            
            # Journaliser les features importantes (pour les modèles qui le supportent)
            if hasattr(best_model, 'coef_'):
                # Récupérer les features les plus importantes
                if isinstance(best_model, LogisticRegression) or isinstance(best_model, LinearSVC):
                    coefs = best_model.coef_[0]
                    if vectorizer_name == "TF-IDF":
                        feature_names = tfidf_vectorizer.get_feature_names_out()
                    else:
                        feature_names = bow_vectorizer.get_feature_names_out()
                    
                    # Créer un DataFrame avec les coefs et les noms des features
                    coefs_df = pd.DataFrame({
                        'feature': feature_names,
                        'importance': coefs
                    })
                    
                    # Trier par importance absolue
                    coefs_df['abs_importance'] = abs(coefs_df['importance'])
                    coefs_df = coefs_df.sort_values('abs_importance', ascending=False).head(20)
                    
                    # Journaliser les features les plus importantes
                    top_features_path = "top_features.csv"
                    coefs_df.to_csv(top_features_path, index=False)
                    mlflow.log_artifact(top_features_path)
                    
                    # Créer un graphique pour visualiser les features les plus importantes
                    plt.figure(figsize=(10, 8))
                    sns.barplot(x='importance', y='feature', data=coefs_df.sort_values('importance', ascending=False).head(20))
                    plt.title(f'Top 20 features positives')
                    plt.tight_layout()
                    mlflow.log_figure(plt.gcf(), "top_positive_features.png")
                    plt.close()
                    
                    plt.figure(figsize=(10, 8))
                    sns.barplot(x='importance', y='feature', data=coefs_df.sort_values('importance').head(20))
                    plt.title(f'Top 20 features négatives')
                    plt.tight_layout()
                    mlflow.log_figure(plt.gcf(), "top_negative_features.png")
                    plt.close()
            
            # Stocker les résultats pour comparaison
            results.append({
                "Modèle": model_name.replace("_", " "),
                "Vectorisation": vectorizer_name,
                "Meilleurs paramètres": str(best_params),
                "Accuracy": acc,
                "Precision": prec,
                "Recall": rec,
                "F1 Score": f1,
                "F2 Score": f2,
                "ROC AUC": roc_auc,
                "Temps d'entraînement (s)": training_time
            })
            
            # Afficher les résultats
            print(f"Meilleurs paramètres: {best_params}")
            print(f"Meilleur score CV (F2): {grid_search.best_score_:.4f}")
            print(f"F1 Score sur test: {f1:.4f}")
            print(f"F2 Score sur test: {f2:.4f}")
            print(f"ROC AUC sur test: {roc_auc:.4f}")
            print(f"Temps d'entraînement: {training_time:.2f} secondes")


        # À la fin de chaque itération, mettez à jour la barre de progression
        progress_bar.update(1)
        progress_bar.set_description(f"Dernier modèle: {model_name} avec {vectorizer_name}")

progress_bar.close()

In [ ]:
# Créer un tableau récapitulatif des résultats
results_df = pd.DataFrame(results)
results_df

In [ ]:
# Créer un run spécial pour le récapitulatif
with mlflow.start_run(run_name="Modele_Simple_Recapitulatif"):
    # Journaliser le tableau des résultats
    results_path = "model_comparison_results.csv"
    results_df.to_csv(results_path, index=False)
    mlflow.log_artifact(results_path)
    
    # Créer et journaliser un graphique de comparaison des performances F1
    plt.figure(figsize=(14, 8))
    plot_data_f1 = results_df.pivot(index='Modèle', columns='Vectorisation', values='F1 Score')
    ax_f1 = plot_data_f1.plot(kind='bar', rot=0)
    plt.title('Comparaison des modèles et vectorisations (F1 Score)')
    plt.ylabel('F1 Score')
    plt.ylim(0.6, 1.0)
    plt.legend(title='Vectorisation')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    for container in ax_f1.containers:
        ax_f1.bar_label(container, fmt='%.3f', padding=3)
    plt.tight_layout()
    mlflow.log_figure(plt.gcf(), "model_comparison_f1.png")
    plt.close()
    
    # Créer et journaliser un graphique de comparaison des performances F2
    plt.figure(figsize=(14, 8))
    plot_data_f2 = results_df.pivot(index='Modèle', columns='Vectorisation', values='F2 Score')
    ax_f2 = plot_data_f2.plot(kind='bar', rot=0)
    plt.title('Comparaison des modèles et vectorisations (F2 Score)')
    plt.ylabel('F2 Score')
    plt.ylim(0.6, 1.0)
    plt.legend(title='Vectorisation')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    for container in ax_f2.containers:
        ax_f2.bar_label(container, fmt='%.3f', padding=3)
    plt.tight_layout()
    mlflow.log_figure(plt.gcf(), "model_comparison_f2.png")
    plt.close()
    
    # Créer et journaliser un graphique de comparaison des performances ROC AUC
    plt.figure(figsize=(14, 8))
    plot_data_roc = results_df.pivot(index='Modèle', columns='Vectorisation', values='ROC AUC')
    ax_roc = plot_data_roc.plot(kind='bar', rot=0)
    plt.title('Comparaison des modèles et vectorisations (ROC AUC)')
    plt.ylabel('ROC AUC')
    plt.ylim(0.6, 1.0)
    plt.legend(title='Vectorisation')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    for container in ax_roc.containers:
        ax_roc.bar_label(container, fmt='%.3f', padding=3)
    plt.tight_layout()
    mlflow.log_figure(plt.gcf(), "model_comparison_roc_auc.png")
    plt.close()
    
    # Journaliser le meilleur modèle selon F2 (puisque c'est notre métrique principale maintenant)
    best_model_row = results_df.loc[results_df['F2 Score'].idxmax()]
    best_model_name = best_model_row['Modèle'].replace(" ", "_")
    best_vectorizer = best_model_row['Vectorisation']
    
    print(f"\nMeilleur modèle (selon F2): {best_model_row['Modèle']} avec {best_vectorizer}")
    print(f"F1 Score: {best_model_row['F1 Score']:.4f}")
    print(f"F2 Score: {best_model_row['F2 Score']:.4f}")
    print(f"ROC AUC: {best_model_row['ROC AUC']:.4f}")
    print(f"Meilleurs paramètres: {best_model_row['Meilleurs paramètres']}")
    
    # Journaliser des informations sur le meilleur modèle
    mlflow.log_param("best_model", f"{best_model_row['Modèle']} avec {best_vectorizer}")
    mlflow.log_param("best_model_f1", best_model_row['F1 Score'])
    mlflow.log_param("best_model_f2", best_model_row['F2 Score'])
    mlflow.log_param("best_model_roc_auc", best_model_row['ROC AUC'])
    mlflow.log_param("best_model_params", best_model_row['Meilleurs paramètres'])
    
    # Afficher le tableau des résultats
    print("\nRécapitulatif des résultats:")
    display(results_df)

In [ ]:
# Sauvegarder le meilleur modèle pour utilisation future
def save_best_model(results_df):
    best_model_row = results_df.loc[results_df['F2 Score'].idxmax()]  # Utiliser F2 maintenant
    best_model_name = best_model_row['Modèle'].replace(" ", "_")
    best_vectorizer = best_model_row['Vectorisation']
    
    # Charger le meilleur modèle depuis MLflow
    client = MlflowClient()
    
    # Trouver le run correspondant au meilleur modèle
    runs = client.search_runs(
        experiment_ids=[mlflow.get_experiment_by_name("OC Projet 7").experiment_id],
        filter_string=f"tags.mlflow.runName = 'Modele_Simple_{best_model_name}_{best_vectorizer}'"
    )
    
    if runs:
        best_run = runs[0]
        run_id = best_run.info.run_id
        
        # Charger le modèle depuis MLflow
        model_uri = f"runs:/{run_id}/model"
        best_model = mlflow.sklearn.load_model(model_uri)
        
        # Charger le vectoriseur depuis les artefacts
        client.download_artifacts(run_id, f"vectorizer_{best_vectorizer}.pkl", "./")
        with open(f"./vectorizer_{best_vectorizer}.pkl", "rb") as f:
            vectorizer = pickle.load(f)
        
        # Créer le dossier de modèles s'il n'existe pas
        os.makedirs("./content/models", exist_ok=True)
        
        # Sauvegarder le modèle
        model_path = f"./content/models/best_model_{best_model_name}_{best_vectorizer}.pkl"
        with open(model_path, "wb") as f:
            pickle.dump(best_model, f)
        
        # Sauvegarder le vectoriseur
        vectorizer_path = f"./content/models/vectorizer_{best_vectorizer}.pkl"
        with open(vectorizer_path, "wb") as f:
            pickle.dump(vectorizer, f)
        
        print(f"Meilleur modèle ({best_model_row['Modèle']} avec {best_vectorizer}) sauvegardé dans le dossier './content/models'")
        print(f"Chemin du modèle: {model_path}")
        print(f"Chemin du vectoriseur: {vectorizer_path}")
        
        # Test du modèle chargé
        if best_vectorizer == "TF-IDF":
            X_test_vec = X_test_tfidf
        else:
            X_test_vec = X_test_bow
            
        y_pred = best_model.predict(X_test_vec)
        f1 = f1_score(y_test, y_pred)
        f2 = fbeta_score(y_test, y_pred, beta=2)
        print(f"F1 Score du meilleur modèle sauvegardé: {f1:.4f}")
        print(f"F2 Score du meilleur modèle sauvegardé: {f2:.4f}")
        
        return best_model, vectorizer, best_model_row['Modèle'], best_vectorizer
    else:
        print("Aucun run trouvé pour le meilleur modèle!")
        return None, None, None, None

In [ ]:
# Sauvegarder le meilleur modèle
best_model, vectorizer, best_model_name, best_vectorizer = save_best_model(results_df)

In [ ]:
# Fonction de prédiction pour l'API
def predict_sentiment(text, model=best_model, vect=vectorizer):
    """
    Fonction qui prend un texte en entrée et retourne la prédiction du sentiment
    Cette fonction pourra être utilisée dans l'API
    """
    # Prétraiter le texte
    processed_text = preprocess_tweet(text)
    
    # Vectoriser
    text_vectorized = vect.transform([processed_text])
    
    # Prédire
    prediction = model.predict(text_vectorized)[0]
    
    # Récupérer la probabilité si le modèle le permet
    if hasattr(model, 'predict_proba'):
        probas = model.predict_proba(text_vectorized)[0]
        confidence = probas[1] if prediction == 1 else probas[0]
    else:
        # Pour les modèles sans predict_proba, comme SVM
        decision = model.decision_function(text_vectorized)[0] if hasattr(model, 'decision_function') else 0
        confidence = abs(decision) / 2  # Normaliser d'une certaine façon
    
    return {
        'sentiment': 'positif' if prediction == 1 else 'négatif',
        'score': float(confidence),
        'model': best_model_name,
        'vectorizer': best_vectorizer
    }

Ajouter des emojis, caractères spéciaux, ponctuations, fautes d'orthographe, langage internet (lol, wtf)

In [ ]:
# Tester la fonction de prédiction sur quelques exemples
def test_model_on_examples(model, vectorizer):
    # Quelques exemples de tweets pour tester le modèle
    test_tweets = [
        "I love this flight, the service was amazing! #happy",
        "Worst flight ever, delayed for 3 hours and no explanation @airline",
        "Nice plane but the food was not good enough.",
        "The staff was helpful but the seats were uncomfortable for a long flight",
        "Amazing experience with Air Paradis today! Great customer service!"
    ]
    
    print("\nTest du meilleur modèle sur quelques exemples:")
    for i, tweet in enumerate(test_tweets):
        result = predict_sentiment(tweet, model, vectorizer)
        print(f"Tweet {i+1}: {tweet}")
        print(f"Sentiment prédit: {result['sentiment']} (score de confiance: {result['score']:.4f})\n")

# Exécuter les tests si le modèle a été correctement chargé
if best_model is not None and vectorizer is not None:
    test_model_on_examples(best_model, vectorizer)

In [ ]:
# Conclusion et étapes suivantes
print("""
## Conclusion de l'approche "Modèle Simple"

Nous avons développé plusieurs modèles classiques pour la détection de sentiment dans les tweets,
en utilisant une optimisation des hyperparamètres via GridSearchCV optimisée sur le F2-score
pour privilégier le rappel (capacité à ne pas manquer de tweets négatifs) :

1. Régression Logistique
2. SVM Linéaire
3. Random Forest
4. Naive Bayes

Chaque modèle a été testé avec deux types de vectorisation :
- Sac de mots (BoW)
- TF-IDF

Le meilleur modèle selon le F2-score est {best_model_name} avec la vectorisation {best_vectorizer}, 
atteignant un F2 Score de {best_f2:.4f}, un F1 Score de {best_f1:.4f} et un ROC AUC de {best_roc_auc:.4f}.

Les hyperparamètres optimaux trouvés pour ce modèle sont : {best_params}

### Avantages du workflow implémenté :

1. **Focalisation sur le rappel avec F2-score** :
   - Réduit le risque de manquer des tweets négatifs (bad buzz potentiels)
   - Particulièrement adapté pour l'anticipation des problèmes de réputation

2. **Suivi complet des expériences avec MLflow** :
   - Tracking de toutes les métriques importantes
   - Sauvegarde des modèles et vectoriseurs
   - Visualisations des performances

3. **Optimisation systématique** :
   - Recherche sur grille des meilleurs hyperparamètres
   - Validation croisée pour éviter le surapprentissage
   - Comparaison rigoureuse des performances

4. **Prétraitement adapté aux tweets** :
   - Traitement spécifique des URLs, mentions et hashtags
   - Conservation des négations et autres éléments linguistiques importants
   - Features supplémentaires basées sur l'analyse des données

### Prochaines étapes :

1. **Développer le modèle sur mesure avancé** utilisant des réseaux de neurones et des word embeddings
2. **Tester l'approche avec BERT** pour comparer les performances
3. **Déployer le modèle via une API** pour le rendre accessible à d'autres applications
4. **Mettre en place le suivi MLOps** pour monitorer les performances du modèle en production

Le modèle actuel servira de référence (baseline) pour évaluer les améliorations apportées par 
les approches plus avancées.
""".format(
    best_model_name=best_model_name,
    best_vectorizer=best_vectorizer,
    best_f1=best_model_row['F1 Score'],
    best_f2=best_model_row['F2 Score'],
    best_roc_auc=best_model_row['ROC AUC'],
    best_params=best_model_row['Meilleurs paramètres']
))

## Explications détaillées

Le code complet intègre plusieurs pratiques MLOps avancées:

### 1. Optimisation des hyperparamètres avec GridSearchCV

J'ai implémenté GridSearchCV pour chaque combinaison de modèle et vectoriseur. Cela permet d'explorer systématiquement l'espace des hyperparamètres pour trouver la configuration optimale. Pour chaque modèle, j'ai défini une grille de paramètres pertinents:

- **Régression Logistique**: différentes valeurs de régularisation (C), types de solveurs
- **SVM**: différentes valeurs de C, options de dualité
- **Random Forest**: nombre d'arbres, profondeur maximale, critères de division
- **Naive Bayes**: différentes valeurs pour le paramètre de lissage alpha

L'optimisation est effectuée avec validation croisée à 5 plis (5-fold) pour garantir la robustesse des résultats.

### 2. Tracking complet avec MLflow

Pour chaque expérience, j'enregistre dans MLflow:

- **Paramètres**: tous les hyperparamètres utilisés, y compris les meilleurs trouvés par GridSearchCV
- **Métriques**: accuracy, precision, recall, F1 score, ROC AUC, temps d'entraînement
- **Artifacts**: 
  - Le modèle optimisé avec sa signature d'entrée/sortie
  - Le vectoriseur utilisé (nouveauté demandée)
  - La matrice de confusion et la courbe ROC
  - Les résultats détaillés de la validation croisée
  - Les features les plus importantes (pour les modèles qui le permettent)
  - Des graphiques d'analyse de l'influence des hyperparamètres

### 3. Récapitulatif global et analyse comparative

Un run spécial "Recapitulatif" est créé pour comparer tous les modèles:

- Tableau comparatif avec toutes les métriques
- Graphiques de comparaison pour F1 Score et ROC AUC
- Identification et journalisation du meilleur modèle

### 4. Sauvegarde et utilisation du meilleur modèle

J'ai implémenté une méthode sophistiquée pour:

1. Identifier le meilleur modèle dans l'historique MLflow
2. Le récupérer avec son vectoriseur
3. Sauvegarder les deux dans un dossier local spécifié ("./content/models")
4. Créer une fonction de prédiction prête pour l'API

### 5. Visualisations avancées

Pour faciliter l'analyse:

- Matrices de confusion pour chaque modèle
- Courbes ROC avec AUC calculé
- Graphiques des features les plus importantes
- Graphiques d'évolution des scores en fonction des hyperparamètres

### Avantages de cette approche

1. **Reproductibilité**: Tous les détails des expériences sont enregistrés
2. **Traçabilité**: Tu peux facilement suivre l'évolution des performances
3. **Maintenabilité**: Le code est modulaire et bien documenté
4. **Optimisation automatique**: La recherche des meilleurs hyperparamètres est systématique
5. **Facilité d'intégration**: Le modèle et vectoriseur sont sauvegardés prêts pour le déploiement

Cette implémentation suit les bonnes pratiques MLOps et pose des bases solides pour les étapes suivantes du projet, notamment le développement de modèles plus avancés et leur déploiement.

### Prochaines étapes :

1. **Développer le modèle sur mesure avancé** utilisant des réseaux de neurones et des word embeddings
2. **Tester l'approche avec BERT** pour comparer les performances
3. **Déployer le modèle via une API** pour le rendre accessible à d'autres applications
4. **Mettre en place le suivi MLOps** pour monitorer les performances du modèle en production